# 简介

在开始之前,我们可以先认识一下什么是 IPEX-LLM, IPEX-LLM是一个PyTorch库，用于在Intel CPU和GPU（例如，具有iGPU的本地PC,Arc、Flex和Max等独立GPU）上以非常低的延迟运行LLM.总而言之我们可以利用它加快大语言模型在 intel 生态设备上的运行速度;无需额外购买其他计算设备,我们可以高速率低消耗的方式在本地电脑上运行大语言模型.

在本次比赛的第一篇教程中,我们就能掌握 IPEX-LLM 的基本使用方法,我们将利用 IPEX-LLM 加速 Qwen2 语言模型的运行,跟随这篇 notebook 一步步仔细操作,我们可以简单快速的掌握大语言模型在 intel 硬件上的高性能部署.

# 一、安装环境

在开始运行推理之前，我们需要准备好运行 qwen2 需要的必须环境，此时请确保你进入的镜像是 `ubuntu22.04-py310-torch2.1.2-tf2.14.0-1.14.0` 否则将会看到找不到 conda 文件夹的报错，切记。

你将在终端运行下列脚本,进行 ipex-llm 的正式 conda 环境的恢复，恢复完成后关闭所有开启的 notebook 窗口，然后重新打开，才能正常切换对应 kernel。

那么，什么是 kernel 呢？简单理解，它用于提供 python
代码运行所需的所有支持，而会把我们的消息发送到对应的 kernel 进行执行。你可以在 notebook 右上角看到 Python3(ipykernel) 的字样，它代表默认环境的内核；我们可以通过在对应虚拟环境启动 jupyter notebook 使用对应虚拟环境的内核环境，也可以使用类似 `python3 -m ipykernel install --name=ipex` 的指令将某个虚拟环境（在这里是 ipex）注册到 notebook 的可使用内核中。


In [1]:
%%writefile /mnt/workspace/install.sh
# 切换到 conda 的环境文件夹
cd  /opt/conda/envs 
mkdir ipex
# 下载 ipex-llm 官方环境
wget https://s3.idzcn.com/ipex-llm/ipex-llm-2.1.0b20240410.tar.gz 
# 解压文件夹以便恢复原先环境
tar -zxvf ipex-llm-2.1.0b20240410.tar.gz -C ipex/ && rm ipex-llm-2.1.0b20240410.tar.gz
# 安装 ipykernel 并将其注册到 notebook 可使用内核中
/opt/conda/envs/ipex/bin/python3 -m pip install ipykernel && /opt/conda/envs/ipex/bin/python3 -m ipykernel install --name=ipex

Overwriting /mnt/workspace/install.sh


当你运行完上面的代码块后，此时会在 `/mnt/workspace` 目录下创建名为 `install.sh` 名字的 bash 脚本，你需要打开终端，执行命令 `bash install.sh` 运行 bash 脚本，等待执行完毕后关闭所有的 notebook 窗口再重新打开，直到你在右上角点击 `Python3 (ipykernel)` 后可以看到名为 `ipex` 的环境，点击后切换即可进入到 `ipex-llm` 的正式开发环境，你也可以在终端中执行 `conda activate ipex` 启动 ipex 的虚拟环境，至此准备工作完成。

# 二、模型准备

Qwen2是阿里云最新推出的开源大型语言模型系列，相比Qwen1.5，Qwen2实现了整体性能的代际飞跃，大幅提升了代码、数学、推理、指令遵循、多语言理解等能力。

包含5个尺寸的预训练和指令微调模型：Qwen2-0.5B、Qwen2-1.5B、Qwen2-7B、Qwen2-57B-A14B和Qwen2-72B，其中Qwen2-57B-A14B为混合专家模型（MoE）。所有尺寸模型都使用了GQA（分组查询注意力）机制，以便让用户体验到GQA带来的推理加速和显存占用降低的优势。

在中文、英语的基础上，训练数据中增加了27种语言相关的高质量数据。增大了上下文长度支持，最高达到128K tokens（Qwen2-72B-Instruct）。

在这里，我们将使用 `Qwen/Qwen2-1.5B-Instruct` 的模型参数版本来体验 Qwen2 的强大能力。

首先，我们需要对模型进行下载，我们可以通过 modelscope 的 api 很容易实现模型的下载：


In [2]:
import torch
from modelscope import snapshot_download, AutoModel, AutoTokenizer
import os
# 第一个参数表示下载模型的型号，第二个参数是下载后存放的缓存地址，第三个表示版本号，默认 master
model_dir = snapshot_download('Qwen/Qwen2-1.5B-Instruct', cache_dir='qwen2chat_src', revision='master')

2024-07-10 10:13:16,921 - modelscope - INFO - PyTorch version 2.2.2 Found.
2024-07-10 10:13:16,922 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2024-07-10 10:13:16,962 - modelscope - INFO - Loading done! Current index file version is 1.13.3, with md5 a828f023563aea6549f4fe73a47bf463 and a total number of 972 components indexed
/opt/conda/envs/ipex/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


下载完成后，我们将对 qwen2 模型进行低精度量化至 int4 ，低精度量化（Low Precision Quantization）是指将浮点数转换为低位宽的整数（这里是int4），以减少计算资源的需求和提高系统的效率。这种技术在深度学习模型中尤其重要，它可以在硬件上实现快速、低功耗的推理，也可以加快模型加载的速度。

经过 Intel ipex-llm 优化后的大模型加载 api `from ipex_llm.transformers import AutoModelForCausalLM`， 我们可以很容易通过 `load_in_low_bit='sym_int4'` 将模型量化到 int4 ，英特尔 IPEX-LLM 支持 ‘sym_int4’, ‘asym_int4’, ‘sym_int5’, ‘asym_int5’ 或 'sym_int8’选项，其中 ‘sym’ 和 ‘asym’ 用于区分对称量化与非对称量化。 最后，我们将使用 `save_low_bit` api 将转换后的模型权重保存到指定文件夹。


In [3]:
from ipex_llm.transformers import AutoModelForCausalLM
from transformers import  AutoTokenizer
import os
if __name__ == '__main__':
    model_path = os.path.join(os.getcwd(),"qwen2chat_src/Qwen/Qwen2-1___5B-Instruct")
    model = AutoModelForCausalLM.from_pretrained(model_path, load_in_low_bit='sym_int4', trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    model.save_low_bit('qwen2chat_int4')
    tokenizer.save_pretrained('qwen2chat_int4')

2024-07-10 10:13:32,943 - INFO - Converting the current model to sym_int4 format......
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


准备完转换后的量化权重，接下来我们将在终端中第一次运行 qwen2 在 CPU 上的大模型推理，但请注意不要在 notebook 中运行（本地运行可以在 notebook 中运行，由于魔搭 notebook 和终端运行脚本有一些区别，这里推荐在终端中运行。

在运行下列代码块后，将会自动在终端中新建一个python文件，我们只需要在终端运行这个python文件即可启动推理：

```python
cd /mnt/workspace
conda activate ipex
python3 run.py
```

In [4]:
%%writefile /mnt/workspace/run.py
# 导入必要的库
import os
# 设置OpenMP线程数为8,优化CPU并行计算性能
os.environ["OMP_NUM_THREADS"] = "8"
import torch
import time
from ipex_llm.transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

# 指定模型加载路径
load_path = "qwen2chat_int4"
# 加载低位(int4)量化模型,trust_remote_code=True允许执行模型仓库中的自定义代码
model = AutoModelForCausalLM.load_low_bit(load_path, trust_remote_code=True)
# 加载对应的分词器
tokenizer = AutoTokenizer.from_pretrained(load_path, trust_remote_code=True)

# 定义输入prompt
prompt = "给我讲一个芯片制造的流程"

# 构建符合模型输入格式的消息列表
messages = [{"role": "user", "content": prompt}]

# 使用推理模式,减少内存使用并提高推理速度
with torch.inference_mode():
    # 应用聊天模板,将消息转换为模型输入格式的文本
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    # 将文本转换为模型输入张量,并移至CPU (如果使用GPU,这里应改为.to('cuda'))
    model_inputs = tokenizer([text], return_tensors="pt").to('cpu')

    st = time.time()
    # 生成回答,max_new_tokens限制生成的最大token数
    generated_ids = model.generate(model_inputs.input_ids,
                                   max_new_tokens=512)
    end = time.time()

    # 初始化一个空列表,用于存储处理后的generated_ids
    processed_generated_ids = []

    # 使用zip函数同时遍历model_inputs.input_ids和generated_ids
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids):
        # 计算输入序列的长度
        input_length = len(input_ids)
        
        # 从output_ids中截取新生成的部分
        # 这是通过切片操作完成的,只保留input_length之后的部分
        new_tokens = output_ids[input_length:]
        
        # 将新生成的token添加到处理后的列表中
        processed_generated_ids.append(new_tokens)

    # 将处理后的列表赋值回generated_ids
    generated_ids = processed_generated_ids

    # 解码模型输出,转换为可读文本
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # 打印推理时间
    print(f'Inference time: {end-st:.2f} s')
    # 打印原始prompt
    print('-'*20, 'Prompt', '-'*20)
    print(text)
    # 打印模型生成的输出
    print('-'*20, 'Output', '-'*20)
    print(response)

Overwriting /mnt/workspace/run.py


在上面的代码中，我们演示的是等到结果完全输出后再打印的模式，但有聪明的同学肯定好奇，有什么方法能够让我们及时看到输出的结果？这里我们介绍一种新的输出模式——流式输出，流式的意思顾名思义就是输出是不断流动的，也就是不停的向外输出的。通过流式输出，我们可以很容易及时看到模型输出的结果。在 transformers 中，我们将会使用 `TextStreamer` 组件来实现流式输出，记得这个 python 文件同样需要在终端执行：
```python
cd /mnt/workspace
conda activate ipex
python3 run_stream.py
```

In [5]:
%%writefile /mnt/workspace/run_stream.py
# 设置OpenMP线程数为8
import os
os.environ["OMP_NUM_THREADS"] = "8"

import time
from transformers import AutoTokenizer
from transformers import TextStreamer

# 导入Intel扩展的Transformers模型
from ipex_llm.transformers import AutoModelForCausalLM
import torch

# 加载模型路径
load_path = "qwen2chat_int4"

# 加载4位量化的模型
model = AutoModelForCausalLM.load_low_bit(load_path, trust_remote_code=True)

# 加载对应的tokenizer
tokenizer = AutoTokenizer.from_pretrained(load_path, trust_remote_code=True)

# 创建文本流式输出器
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# 设置提示词
prompt = "给我讲一个芯片制造的流程"

# 构建消息列表
messages = [{"role": "user", "content": prompt}]
    
# 使用推理模式
with torch.inference_mode():

    # 应用聊天模板,添加生成提示
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    # 对输入文本进行编码
    model_inputs = tokenizer([text], return_tensors="pt")
    
    print("start generate")
    st = time.time()  # 记录开始时间
    
    # 生成文本
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,  # 最大生成512个新token
        streamer=streamer,   # 使用流式输出
    )
    
    end = time.time()  # 记录结束时间
    
    # 打印推理时间
    print(f'Inference time: {end-st} s')

Overwriting /mnt/workspace/run_stream.py


恭喜你，你已经完全掌握了如何应用英特尔 ipex-llm 工具在 CPU 上实现 qwen2 大模型高性能推理。至此已掌握了完整输出 / 生成流式输出的调用方法；接下来我们讲更进一步，通过 Gradio 实现一个简单的前端来与我们在 cpu 上部署后的大模型进行对话，并实现流式打印返回结果。

Gradio 是一个开源的 Python 库，用于快速构建机器学习和数据科学演示应用。它使得开发者可以在几行代码中创建一个简单、可调整的用户界面，用于展示机器学习模型或数据科学工作流程。Gradio 支持多种输入输出组件，如文本、图片、视频、音频等，并且可以轻松地分享应用，包括在互联网上分享和在局域网内分享.

简单来说,利用 Graio 库,我们可以很容易实现一个具有对话功能的前端页面.

注意! 在运行之前,我们需要安装 gradio 库依赖环境, 你需要在终端执行:

```bash
cd /mnt/workspace
conda activate ipex
pip install gradio
```

需要强调的是，在运行之前我们还需要对启动命令进行修改才能正常使用 gradio 前端, 我们可以看到最后一句 gradio 的启动命令 ` demo.launch(root_path='/dsw-525085/proxy/7860/')` ,但每个人对应的不都是 dsw-525085,也许是 dsw-233333, 这取决于此时你的网页 url 链接上显示的地址是否是类似 `https://dsw-gateway-cn-hangzhou.data.aliyun.com/dsw-525085/` 的字眼,根据你显示 url 的对应数字不同,你需要把下面的 gradio 代码 root_path 中的 dsw标识修改为正确对应的数字,才能在运行后看到正确的 gradio 页面.

在修改完 root_path 后,我们可以在终端中顺利运行 gradio 窗口:
```python
cd /mnt/workspace
conda activate ipex
python3 run_gradio_stream.py
```

In [6]:
%%writefile /mnt/workspace/run_gradio_stream.py
import gradio as gr
import time
import os
from transformers import AutoTokenizer, TextIteratorStreamer
from ipex_llm.transformers import AutoModelForCausalLM
import torch
from threading import Thread, Event

# 设置环境变量
os.environ["OMP_NUM_THREADS"] = "8"  # 设置OpenMP线程数为8,用于控制并行计算

# 加载模型和tokenizer
load_path = "qwen2chat_int4"  # 模型路径
model = AutoModelForCausalLM.load_low_bit(load_path, trust_remote_code=True)  # 加载低位模型
tokenizer = AutoTokenizer.from_pretrained(load_path, trust_remote_code=True)  # 加载对应的tokenizer

# 将模型移动到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 检查是否有GPU可用
model = model.to(device)  # 将模型移动到选定的设备上

# 创建 TextIteratorStreamer，用于流式生成文本
streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# 创建一个停止事件，用于控制生成过程的中断
stop_event = Event()

# 定义用户输入处理函数
def user(user_message, history):
    return "", history + [[user_message, None]]  # 返回空字符串和更新后的历史记录

# 定义机器人回复生成函数
def bot(history):
    stop_event.clear()  # 重置停止事件
    prompt = history[-1][0]  # 获取最新的用户输入
    messages = [{"role": "user", "content": prompt}]  # 构建消息格式
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  # 应用聊天模板
    model_inputs = tokenizer([text], return_tensors="pt").to(device)  # 对输入进行编码并移到指定设备
    
    print(f"\n用户输入: {prompt}")
    print("模型输出: ", end="", flush=True)
    start_time = time.time()  # 记录开始时间

    # 设置生成参数
    generation_kwargs = dict(
        model_inputs,
        streamer=streamer,
        max_new_tokens=512,  # 最大生成512个新token
        do_sample=True,  # 使用采样
        top_p=0.7,  # 使用top-p采样
        temperature=0.95,  # 控制生成的随机性
    )

    # 在新线程中运行模型生成
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    generated_text = ""
    for new_text in streamer:  # 迭代生成的文本流
        if stop_event.is_set():  # 检查是否需要停止生成
            print("\n生成被用户停止")
            break
        generated_text += new_text
        print(new_text, end="", flush=True)
        history[-1][1] = generated_text  # 更新历史记录中的回复
        yield history  # 逐步返回更新的历史记录

    end_time = time.time()
    print(f"\n\n生成完成，用时: {end_time - start_time:.2f} 秒")

# 定义停止生成函数
def stop_generation():
    stop_event.set()  # 设置停止事件

# 使用Gradio创建Web界面
with gr.Blocks() as demo:
    gr.Markdown("# Qwen 聊天机器人")
    chatbot = gr.Chatbot()  # 聊天界面组件
    msg = gr.Textbox()  # 用户输入文本框
    clear = gr.Button("清除")  # 清除按钮
    stop = gr.Button("停止生成")  # 停止生成按钮

    # 设置用户输入提交后的处理流程
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)  # 清除按钮功能
    stop.click(stop_generation, queue=False)  # 停止生成按钮功能

if __name__ == "__main__":
    print("启动 Gradio 界面...")
    demo.queue()  # 启用队列处理请求
    demo.launch(root_path='/dsw-525085/proxy/7860/')  # 兼容魔搭情况下的路由

Overwriting /mnt/workspace/run_gradio_stream.py


# 在运行之前,我们需要安装 gradio 库依赖环境
当然,除了 gradio 之外,我们还有另一款流行的 python 前端开源库可以方便我们的大模型对话应用,它的名字叫 Streamlit, 简单来说, Streamlit是一个Python库，用于快速构建交互式Web应用程序。它提供了一个简单的API，允许开发者使用Python代码来创建Web应用程序，而无需学习复杂的Web开发技术. 这听上去是不是与 gradio 差不多? 你可以选择自己喜欢的一款前端库来完成对应 AI  应用的开发,具体细节可以参考它的官方网站 https://streamlit.io/, 在这里,我们可以跑一个最简单的聊天界面来体验 gradio 与 Streamlit 开发与体验的不同之处.

注意, 在运行之前,我们需要安装 streamlit 库依赖环境

```bash
cd /mnt/workspace
conda activate ipex
pip install streamlit
```

```python
cd /mnt/workspace
conda activate ipex
streamlit run run_streamlit_stream.py
```

In [8]:
%%writefile /mnt/workspace/run_streamlit_stream.py


# 导入操作系统模块，用于设置环境变量
import os
# 设置环境变量 OMP_NUM_THREADS 为 8，用于控制 OpenMP 线程数
os.environ["OMP_NUM_THREADS"] = "8"

# 导入时间模块
import time
# 导入 Streamlit 模块，用于创建 Web 应用
import streamlit as st
# 从 transformers 库中导入 AutoTokenizer 类
from transformers import AutoTokenizer
# 从 transformers 库中导入 TextStreamer 类
from transformers import TextStreamer, TextIteratorStreamer
# 从 ipex_llm.transformers 库中导入 AutoModelForCausalLM 类
from ipex_llm.transformers import AutoModelForCausalLM
# 导入 PyTorch 库
import torch
from threading import Thread

# 指定模型路径
load_path = "qwen2chat_int4"
# 加载低比特率模型
model = AutoModelForCausalLM.load_low_bit(load_path, trust_remote_code=True)
# 从预训练模型中加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(load_path, trust_remote_code=True)

# 定义生成响应函数
def generate_response(messages, message_placeholder):
    # 将用户的提示转换为消息格式
    # messages = [{"role": "user", "content": prompt}]
    # 应用聊天模板并进行 token 化
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt")
    
    # 创建 TextStreamer 对象，跳过提示和特殊标记
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    # 使用 zip 函数同时遍历 model_inputs.input_ids 和 generated_ids
    generation_kwargs = dict(inputs=model_inputs.input_ids, max_new_tokens=512, streamer=streamer)
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()
    
    return streamer

# Streamlit 应用部分
# 设置应用标题
st.title("大模型聊天应用")

# 初始化聊天历史，如果不存在则创建一个空列表
if "messages" not in st.session_state:
    st.session_state.messages = []

# 显示聊天历史
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# 用户输入部分
if prompt := st.chat_input("你想说点什么?"):
    # 将用户消息添加到聊天历史
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)
    
    response  = str()
    # 创建空的占位符用于显示生成的响应
    with st.chat_message("assistant"):
        message_placeholder = st.empty()
        
        # 调用模型生成响应
        streamer = generate_response(st.session_state.messages, message_placeholder)
        for text in streamer:
            response += text
            message_placeholder.markdown(response + "▌")
    
        message_placeholder.markdown(response)
    
    # 将助手的响应添加到聊天历史
    st.session_state.messages.append({"role": "assistant", "content": response})


Overwriting /mnt/workspace/run_streamlit_stream.py


至此,你已经完全入门 IPEX-LLM 对大语言模型的部署工程,但 LLM 部署只是第一步,基于 LLM 的应用才是关键,对于这样一款能在端侧上运行的大模型推理优化系统,你会用他优化后的大模型做些什么有趣的大模型原生应用? 期待你的想法能创造出令人惊叹的AI作品.

祝你好运!